<a href="https://colab.research.google.com/github/taerri/meal-kit-analysis/blob/main/%EB%B9%85%EC%B9%B4%EC%9D%B8%EC%A6%88_%EC%A0%84%EC%B2%98%EB%A6%ACipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **준비**

In [1]:
pip install konlpy

In [2]:
pip install Counter

In [3]:
pip install customized_konlpy

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from ckonlpy.tag import Twitter
from konlpy.tag import Okt
from konlpy.tag import Komoran
from collections import Counter
from wordcloud import WordCloud

In [5]:
twitter=Twitter()

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


# **하나의 데이터로 만들기**

In [6]:
df1=pd.read_csv("밀키트_20170101_20171231.csv")
df2=pd.read_csv("밀키트_20180101_20180701.csv")
df3=pd.read_csv("밀키트_20180701_20180715.csv")
df4=pd.read_csv("밀키트_20180715_20181231.csv")
df5=pd.read_csv("밀키트_20181231_20190101.csv")
df6=pd.read_csv("밀키트_20190101_20190301.csv")
df7=pd.read_csv("밀키트_20190301_20190601.csv")
df8=pd.read_csv("밀키트_20190601_20190901.csv")
df9=pd.read_csv("밀키트_20190901_20200101.csv")
df10=pd.read_csv("밀키트_20200101_20200401.csv")
df11=pd.read_csv("밀키트_20200401_20200501.csv")
df12=pd.read_csv("밀키트_20200501_20200701.csv")
df13=pd.read_csv("밀키트_20200701_20200801.csv")
df14=pd.read_csv("밀키트_20200801_20200901.csv")
df15=pd.read_csv("밀키트_20200901_20201001.csv")
df16=pd.read_csv("밀키트_20201001_20201101.csv")

In [7]:
#데이터 합치기
df=pd.concat((df1,df2,df3,df4,df5),axis=0)
df.head()

,Date,Title,Content
0,2017-12-27,"연말연시 ‘홈파티족’ 확산, 홈파티 ‘잇츠온’ 간편식 직접 체험해보니",요리하는 재미 ‘쏠쏠~~’ 맛도 ‘일품’ 분위기는 ‘덤’\n\n\n\n거리를 가득 ...
1,2017-12-27,식물성·간편식·슈퍼곡물에 빠졌던 2017년,식품을 낭비없이 먹는 ‘제로 웨이스트’…SNS발달로 이국적 맛 ‘에스닉 푸드’ 등 ...
2,2017-12-27,"올해만 10여개 브랜드 출시, 가정간편식 '춘추전국시대'","야구르트 아줌마가 배달하는 간편식,백화점의 프리미엄 간편식, 편의점에서 간단히 만나..."
3,2017-12-26,유통업계 '밀 솔루션' 경쟁 본격화,유통업계가 소비자 라이프스타일 변화에 발맞춰 간편한 한끼 식사가 가능한 제품과 서비...
4,2017-12-19,"해먹남녀, GS리테일 ‘심플리쿡’ 판매 진행 “칼질 없이 일품요리 즐겨”",푸드 콘텐츠 커머스 플랫폼 ‘해먹남녀’를 운영하는 바이탈힌트 코리아(대표 정지웅)는...


In [8]:
#본문 리스트 만들기
word_list=[]
word_list=df['Content']
word_list=word_list.reset_index(drop=True)
word_list

0      요리하는 재미 ‘쏠쏠~~’ 맛도 ‘일품’ 분위기는 ‘덤’\n\n\n\n거리를 가득 ...
1      식품을 낭비없이 먹는 ‘제로 웨이스트’…SNS발달로 이국적 맛 ‘에스닉 푸드’ 등 ...
2      야구르트 아줌마가 배달하는 간편식,백화점의 프리미엄 간편식, 편의점에서 간단히 만나...
3      유통업계가 소비자 라이프스타일 변화에 발맞춰 간편한 한끼 식사가 가능한 제품과 서비...
4      푸드 콘텐츠 커머스 플랫폼 ‘해먹남녀’를 운영하는 바이탈힌트 코리아(대표 정지웅)는...
                             ...                        
448    현대백화점의 식품 전문 온라인몰인 ‘e슈퍼마켓’은 백화점 업계 최초로 CJ대한통운과...
449    미국의 대형 유통업체들이 밀키트(Meal Kits) 시장에 눈독을 들이고 있다. 밀...
450    [아시아경제 이선애 기자] 밀레니얼 세대가 성장해 가정을 꾸리기 시작하면서 각 가정...
451    [디지털타임스 김아름 기자] 가정간편식(HMR)의 성장세에 외식업계가 고민에 빠졌다...
452    ◀ 앵커 ▶\n\n경제 소식, 쉽고 재미있게 전해드리는 <경제, 한 눈에 쏙> 시간...
Name: Content, Length: 453, dtype: object

In [9]:
#기사중복 삭제
df_drop=df.drop_duplicates(['Date','Title','Content'], keep='first')

In [10]:
df_drop

,Date,Title,Content
0,2017-12-27,"연말연시 ‘홈파티족’ 확산, 홈파티 ‘잇츠온’ 간편식 직접 체험해보니",요리하는 재미 ‘쏠쏠~~’ 맛도 ‘일품’ 분위기는 ‘덤’\n\n\n\n거리를 가득 ...
1,2017-12-27,식물성·간편식·슈퍼곡물에 빠졌던 2017년,식품을 낭비없이 먹는 ‘제로 웨이스트’…SNS발달로 이국적 맛 ‘에스닉 푸드’ 등 ...
2,2017-12-27,"올해만 10여개 브랜드 출시, 가정간편식 '춘추전국시대'","야구르트 아줌마가 배달하는 간편식,백화점의 프리미엄 간편식, 편의점에서 간단히 만나..."
3,2017-12-26,유통업계 '밀 솔루션' 경쟁 본격화,유통업계가 소비자 라이프스타일 변화에 발맞춰 간편한 한끼 식사가 가능한 제품과 서비...
4,2017-12-19,"해먹남녀, GS리테일 ‘심플리쿡’ 판매 진행 “칼질 없이 일품요리 즐겨”",푸드 콘텐츠 커머스 플랫폼 ‘해먹남녀’를 운영하는 바이탈힌트 코리아(대표 정지웅)는...
...,...,...,...
227,2018-07-17 이선애,한국야쿠르트 HMR '잇츠온' 출시 1년만에 345만개 판매,[아시아경제 이선애 기자]한국야쿠르트 가정간편식(HMR) 브랜드 ‘잇츠온’이 출시 ...
228,2018-07-16 석유선,"[위기의 유통업, 大반전 키워드④ 신선식품] 더 빠르고 싱싱하게…‘새벽배송 전쟁’",현대백화점의 식품 전문 온라인몰인 ‘e슈퍼마켓’은 백화점 업계 최초로 CJ대한통운과...
229,2018-07-16 육성연 기자,[aT와 함께하는 글로벌푸드 리포트] 美 간편식 등 인기…‘밀키트’시장 가파른 성장세,미국의 대형 유통업체들이 밀키트(Meal Kits) 시장에 눈독을 들이고 있다. 밀...
0,2019-01-01 이선애,"2019년 밀레니얼 가족의 식사 풍경…""틈새끼니족…간편 식사를 잡아라""",[아시아경제 이선애 기자] 밀레니얼 세대가 성장해 가정을 꾸리기 시작하면서 각 가정...


In [11]:
#사전에 단어 추가
twitter.add_dictionary('밀키트', 'Noun')

In [12]:
#단어 교체
df['Content']=df['Content'].map(lambda x: str(x).replace('밀 키트','밀키트'))

# **형태소 분석**

In [13]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
from tqdm import tqdm
kkma = Kkma()

In [15]:
temp = [] 
for x in tqdm(df["Content"]):
    try:
        temp.append(kkma.nouns(x))
    except:
        temp.append(["None"])

100%|██████████| 453/453 [03:46<00:00,  2.00it/s]


In [50]:
konlpy_df= df.copy()
konlpy_df["ContentNoun"] = temp

In [71]:
konlpy_df.to_json("./mealkit_형태소분석결과.json",orient="table")

In [72]:
#형태소 분리
#sentences_tag=[]

#for sentence in word_list:
 #   morph=twitter.nouns(sentence)
  #  sentences_tag.append(morph)
   # print(morph)

# **형태소 분석 된 데이터 읽기**

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
import pickle
import re

In [74]:
corpus = pd.read_json("./mealkit_형태소분석결과.json",orient="table")

In [75]:
tokenized_doc = list(corpus["ContentNoun"])

# **불용어 제거**

In [76]:
stopword1 = pd.read_csv("./한국어불용어100.txt",header=None,sep="\t")
stopword1 = list(stopword1[0])

In [77]:
stopword2 = pd.read_csv("./한국어불용어100_2.txt",header=None)
stopword2 = list(stopword2[0])

In [78]:
# 직접 제거
stopword3 = pd.read_excel("./삭제 단어.xlsx")
stopword3.columns =["단어","불용어여부"]
stopword3 = stopword3[stopword3.불용어여부 == "x"]
stopword3 = list(stopword3["단어"])

In [79]:
stopword = list(set(stopword1 + stopword2 + stopword3))
stopword = [x for x in stopword if not re.compile("[0-9]+").search(str(x))]
print("불용어 개수 :",len(stopword))

불용어 개수 : 664


In [80]:
tokenized_doc = [[x for x in y if x not in stopword] for y in tokenized_doc]

In [81]:
corpus["ContentNoun"] = tokenized_doc

In [82]:
corpus.to_json("./mealkit_전처리.json",orient="table")

In [83]:
corpus["ContentNoun"]

0      [요리, 재미, 맛, 일품, 분위기, 덤, 거리, 인파, 1, 1시간, 기본, 식당...
1      [식품, 낭비, 제로, 웨이스트, 발달, 이국적, 맛, 에스, 에스닉, 닉, 푸드,...
2      [야구르트, 아줌마, 배달, 간편, 간편식, 식, 백화점, 프리미엄, 간, 편식, ...
3      [유통, 유통업계, 업계, 소비자, 라이프, 라이프스타일, 스타일, 변화, 끼, 식...
4      [푸드, 콘텐츠, 머스, 플랫폼, 해먹, 해먹남녀, 남녀, 운영, 바이탈, 바이탈힌...
                             ...                        
448    [현대, 현대백화점, 백화점, 식품, 전문, 온라인, 온라인몰, 몰, 슈퍼마켓, 은...
449    [미국, 대형, 유통, 유통업체, 업체, 키트, 시장, 눈독, 밀, 밀키트, 성장세...
450    [아시아, 아시아경제, 경제, 선, 선애, 애, 기자, 밀레, 얼, 세대, 성장, ...
451    [디지털, 디지털타임스, 타임스, 김, 김아름, 아름, 기자, 가정, 가정간편식, ...
452    [앵커, 경제, 소식, 눈, 경제부, 이지선, 기자, 오늘, 2018, 2018년,...
Name: ContentNoun, Length: 453, dtype: object

In [84]:
corpus.head()

,index,Date,Title,Content,ContentNoun
0,0,2017-12-27,"연말연시 ‘홈파티족’ 확산, 홈파티 ‘잇츠온’ 간편식 직접 체험해보니",요리하는 재미 ‘쏠쏠~~’ 맛도 ‘일품’ 분위기는 ‘덤’\n\n\n\n거리를 가득 ...,"[요리, 재미, 맛, 일품, 분위기, 덤, 거리, 인파, 1, 1시간, 기본, 식당..."
1,1,2017-12-27,식물성·간편식·슈퍼곡물에 빠졌던 2017년,식품을 낭비없이 먹는 ‘제로 웨이스트’…SNS발달로 이국적 맛 ‘에스닉 푸드’ 등 ...,"[식품, 낭비, 제로, 웨이스트, 발달, 이국적, 맛, 에스, 에스닉, 닉, 푸드,..."
2,2,2017-12-27,"올해만 10여개 브랜드 출시, 가정간편식 '춘추전국시대'","야구르트 아줌마가 배달하는 간편식,백화점의 프리미엄 간편식, 편의점에서 간단히 만나...","[야구르트, 아줌마, 배달, 간편, 간편식, 식, 백화점, 프리미엄, 간, 편식, ..."
3,3,2017-12-26,유통업계 '밀 솔루션' 경쟁 본격화,유통업계가 소비자 라이프스타일 변화에 발맞춰 간편한 한끼 식사가 가능한 제품과 서비...,"[유통, 유통업계, 업계, 소비자, 라이프, 라이프스타일, 스타일, 변화, 끼, 식..."
4,4,2017-12-19,"해먹남녀, GS리테일 ‘심플리쿡’ 판매 진행 “칼질 없이 일품요리 즐겨”",푸드 콘텐츠 커머스 플랫폼 ‘해먹남녀’를 운영하는 바이탈힌트 코리아(대표 정지웅)는...,"[푸드, 콘텐츠, 머스, 플랫폼, 해먹, 해먹남녀, 남녀, 운영, 바이탈, 바이탈힌..."
